#This is the wrangling code for data used in iteration 1.

In [0]:
#https://towardsdatascience.com/reverse-geocoding-in-python-a915acf29eb6
#install packages
#!pip install ipython-autotime
#!pip install geopandas
#!pip install geopy
#%load_ext autotime

#all libraries used
import pandas as pd
import numpy as np
#import geopandas as gpd
#import geopy
#from geopy.geocoders import Nominatim

#instantiate nominatim as the locator to be used for this notebook
#locator = Nominatim(user_agent="locGeoCoder", timeout=10)

**This section is collect the suburbs and postcode of the collected open data with missing suburb and postcode but coordinates ae available**

In [0]:
#read the file that includes Libraries Vic and Melbourne Building Infor 2018 data
file_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/postcode_test.csv"
df = pd.read_csv(file_path)
#view file structure
df.head()

#combine longitude and latitude as coordinate points (x,y) that will be readable by Nominatim
df["coordinates"] =  df["LAT"].astype(str)+","+df["LONG"].astype(str)

#get suburb and postcode
df["Temp_Suburb"] = df["coordinates"].apply(lambda x: dict(locator.reverse(x).raw).get("address").get("suburb"))
df["Temp_Postcode"] = df["coordinates"].apply(lambda x: dict(locator.reverse(x).raw).get("address").get("postcode"))

time: 8min 48s


In [0]:
#this is to save the file as csv for additional external analysis in excel
#df.to_csv(r'buildinginfo_libraries_data.csv', index = False)
#mount the drive in google collabb

#https://colab.research.google.com/notebooks/io.ipynb
#!cat buildinginfo_libraries_data.csv "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/"

time: 1.53 ms


**This section is collect the address, suburbs and postcode of the collected open data with only longitude and latitude available**

In [0]:
#Getting missing data from McDonald's restaurant data
file_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/maccas_coords_test.csv"
df = pd.read_csv(file_path)
#get relevant columns
df = df.drop("Property ID", axis=1)
#update column names for consistentcy
df.columns = ["LONG","LAT"]
df["coordinates"] =  df["LAT"].astype(str)+","+df["LONG"].astype(str)
df.head()

,LONG,LAT,coordinates
0,144.967849,-37.813145,"-37.81314501,144.9678487"
1,144.963661,-37.814433,"-37.81443332,144.9636607"
2,144.961738,-37.814103,"-37.81410348,144.96173819999998"
3,144.958334,-37.817538,"-37.81753764,144.9583335"
4,144.972707,-37.813419,"-37.81341901,144.9727073"


time: 25.6 ms


In [0]:
#https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns
#function to get address, postcode, and suburb data based on latutude and longitude
def addressPostSub(coords):
  full_add = dict(locator.reverse(coords).raw)

  #get postcode and suburb
  address_fields = full_add.get("address")
  suburb = address_fields.get("suburb")
  postcode = address_fields.get("postcode")

  #remove values from display name as the final address
  county = address_fields.get("county")
  address = full_add.get("display_name").replace(county,"").replace("Australia","").replace("Victoria","").replace(",","").replace("  "," ")
  
  return suburb, address, postcode

time: 8.21 ms


In [0]:
#this collects data for the loaded file
#creats columns using dictionary comprehension
#https://stackoverflow.com/questions/24204087/python-dictionary-get-multiple-values
df['Suburb'], df['Address'], df['Postcode'] = zip(*df['coordinates'].map(addressPostSub))
df

,LONG,LAT,coordinates,Suburb,Address,Postcode
0,144.967849,-37.813145,"-37.81314501,144.9678487",Melbourne City,Bourke House 145 Russell Street Melbourne City...,3000
1,144.963661,-37.814433,"-37.81443332,144.9636607",Melbourne City,London Stores Building Bourke Street Mall Cent...,3000
2,144.961738,-37.814103,"-37.81410348,144.96173819999998",Melbourne City,City Parking 392 Bourke Street Hardware Lane M...,3000
3,144.958334,-37.817538,"-37.81753764,144.9583335",Melbourne City,Collins House Saint James Lane Melbourne City ...,3000
4,144.972707,-37.813419,"-37.81341901,144.9727073",Melbourne City,28 Collins Street Melbourne City 3000,3000
5,144.964500,-37.817829,"-37.81782906,144.96449950000002",Melbourne City,McDonald's Flinders Court Melbourne City 3000,3000
6,144.959552,-37.807352,"-37.80735246,144.9595521",Melbourne City,McDonald's Therry Street Melbourne Innovation ...,3000
7,144.949523,-37.793819,"-37.7938185,144.9495226",Parkville,Royal Children's Hospital 50 Flemington Road P...,3051
8,144.962756,-37.810902,"-37.81090175,144.9627564",Melbourne City,Melbourne Central Menzies Alley Melbourne City...,3000
9,144.976177,-37.836151,"-37.83615131,144.976177",Melbourne City,407 St Kilda Road Melbourne City 3004,3004


time: 13.5 s


In [0]:
#this is to save the file as csv for additional external analysis in excel
#df.to_csv(r'businessbldgs_maccas.csv', index = False)
#mount the drive in google collabb

#https://stackoverflow.com/questions/53898836/export-dataframe-as-csv-file-from-google-colab-to-google-drive
#!cp businessbldgs_maccas.csv "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/"

time: 1.52 ms


**This section is the code to get the latitude and longitude of addresses collected without coordinates**

In [0]:
#this is the file for launch housing and salvation army data
file_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/launch_salvation_test.csv"
df = pd.read_csv(file_path)

time: 1 s


In [0]:
#https://geopy.readthedocs.io/en/stable/
#functions to get the lat or lon, and indicate an error if the coordinates cannot be found
def lat_check(x):
    try:
        return locator.geocode(x).latitude
    except:
        return 'error'

def lon_check(x):
    try:
        return locator.geocode(x).longitude
    except:
        return 'error'

time: 3.43 ms


In [0]:
#Apply the functions above to each row using the addresses in the file that contains launch housing and salvation army data
df["Temp_LAT"] = df["Address"].apply(lambda x: lat_check(x))
df["Temp_LON"] = df["Address"].apply(lambda x: lon_check(x))

time: 20.9 s


In [0]:
#this is to safe the dataframe in csv format for checking v
#df.to_csv(r'/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/LaunchSalvation_data.csv', index = False)
#!cp LaunchSalvation_data.csv "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/"

time: 862 µs


In [0]:
#after inspection, 2 addresses had errors, due to floor level inclusion of the addresses, manually updated in the database
add1 = "11 Chesterville Road Cheltenham VIC 3192"
add2 = "30-540 Springvale Road Glen Waverley VIC 3150"
adds = [add1,add2]
for add in adds:
  print(add)
  print(locator.geocode(add).latitude)
  print(locator.geocode(add).longitude) 

11 Chesterville Road Cheltenham VIC 3192
-37.9523898
145.0581187
30-540 Springvale Road Glen Waverley VIC 3150
-37.8743131
145.1669381
time: 3.9 s


**This section covers McDonald's data collected from google places search and McDonald's website**

In [0]:
file_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/maccas_24hrs_test.csv"
df = pd.read_csv(file_path)
df

,McDonald's Address Search
0,McDonald's Prahran VIC
1,McDonald's Elsternwick VIC
2,McDonald's Burnley Richmond VIC
3,McDonald's Richmond VIC
4,McDonald's Oakleigh VIC
5,McDonald's Holmesglen Malvern East VIC
6,McDonald's Oakleigh South VIC
7,McDonald's St Kilda VIC
8,McDonald's Chapel Street South Yarra VIC
9,McDonald's Burwood VIC


time: 17.3 ms


In [0]:
#This is the function to get the full location details for McDonald's indetified as 24 hrs with establishment names only as displayed on google
def fullLoc(add):
  #get lat and lon based on address name
  full_locator = locator.geocode(add)
  lat = full_locator.latitude
  lon = full_locator.longitude
  #get full name of address
  temp_add = full_locator.address
  
  #get specific fields in the address
  #reverse geocoding to get location details postcode and suburb
  coords = str(lat)+","+str(lon)
  address_fields = dict(locator.reverse(coords).raw).get("address")
  suburb = address_fields.get("suburb")
  postcode = address_fields.get("postcode")
  county = address_fields.get("county")
  address = temp_add.replace(county,"").replace("Australia","").replace("Victoria","").replace(",","").replace("  "," ").replace("  "," ")
  
  return lat, lon, suburb, address, postcode

time: 10.4 ms


In [0]:
#create a new dataframe for columns aligned with the charging station database
df['LAT'],df['LONG'],df['Suburb'], df['Address'], df['Postcode'] = zip(*df['McDonald\'s Address Search'].map(fullLoc))
df

,McDonald's Address Search,LAT,LONG,Suburb,Address,Postcode
0,McDonald's Prahran VIC,-37.848368,145.002879,Prahran,McDonald's Williams Road Prahran 3181,3181
1,McDonald's Elsternwick VIC,-37.884452,144.998217,Elsternwick,McDonald's Nepean Highway Elsternwick 3185,3185
2,McDonald's Burnley Richmond VIC,-37.830049,145.006454,Richmond,Burnley Street Richmond 3121,3121
3,McDonald's Richmond VIC,-37.819604,145.011847,Richmond,McDonald's Bridge Road Richmond 3121,3121
4,McDonald's Oakleigh VIC,-37.900700,145.095674,Oakleigh,Oakleigh,VIC 3166
5,McDonald's Holmesglen Malvern East VIC,-37.875831,145.091004,Malvern East,McDonald's Batesford Road Holmesglen Malvern E...,3148
6,McDonald's Oakleigh South VIC,-37.924826,145.092174,Oakleigh South,Oakleigh South 3167,3167
7,McDonald's St Kilda VIC,-37.866825,144.977662,St Kilda,McDonald's McDonald's Drive Thru St Kilda 3182,3182
8,McDonald's Chapel Street South Yarra VIC,-37.838172,144.995881,South Yarra,Chapel Street South Yarra 3165,3165
9,McDonald's Burwood VIC,-37.850921,145.097727,Burwood,McDonald's 52 Burwood Highway Burwood 3125,3125


time: 34.2 s


,McDonald's Address Search,LAT,LONG,Suburb,Address,Postcode
0,McDonald's Prahran VIC,-37.848368,145.002879,Prahran,McDonald's Williams Road Prahran 3181,3181
1,McDonald's Elsternwick VIC,-37.884452,144.998217,Elsternwick,McDonald's Nepean Highway Elsternwick 3185,3185
2,McDonald's Burnley Richmond VIC,-37.830049,145.006454,Richmond,Burnley Street Richmond 3121,3121
3,McDonald's Richmond VIC,-37.819604,145.011847,Richmond,McDonald's Bridge Road Richmond 3121,3121
4,McDonald's Oakleigh VIC,-37.900700,145.095674,Oakleigh,Oakleigh,VIC 3166
5,McDonald's Holmesglen Malvern East VIC,-37.875831,145.091004,Malvern East,McDonald's Batesford Road Holmesglen Malvern E...,3148
6,McDonald's Oakleigh South VIC,-37.924826,145.092174,Oakleigh South,Oakleigh South 3167,3167
7,McDonald's St Kilda VIC,-37.866825,144.977662,St Kilda,McDonald's McDonald's Drive Thru St Kilda 3182,3182
8,McDonald's Chapel Street South Yarra VIC,-37.838172,144.995881,South Yarra,Chapel Street South Yarra 3165,3165
9,McDonald's Burwood VIC,-37.850921,145.097727,Burwood,McDonald's 52 Burwood Highway Burwood 3125,3125


time: 33.7 s


In [0]:
#this is to save the file as csv for additional external analysis in excel
#df.to_csv(r'maccas_24hrs_data.csv', index = False)
#mount the drive in google collabb

#https://stackoverflow.com/questions/53898836/export-dataframe-as-csv-file-from-google-colab-to-google-drive
#!cp maccas_24hrs_data.csv "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/"

time: 863 µs


**This section is to visualise the data in the database for checking**

In [2]:
file_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/For Upload/04272020_upload_v3.csv"
df = pd.read_csv(file_path)
#view file structure
df.head()

,id,station_name,longitude,latitude,address,suburb,postcode,if_charger_working,usb_a,usb_c,micro_usb,plug_only,establishment_type,if_wifi,if_bathroom,access_type,other_amenities,star_rating,mon_open,mon_close,tue_open,tue_close,wed_open,wed_close,thu_open,thu_close,fri_open,fri_close,sat_open,sat_close,sun_open,sun_close,if_24h
0,1,Northern Pavilion,144.961529,-37.784924,360 Royal Parade Carlton North 3052,Carlton North,3052,1,0,1,1,0,Community Space,2,2,Low level of accessibility,NaN,NaN,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,0
1,2,Sydenham,144.951859,-37.796409,10 Gatehouse Street Parkville 3052,Parkville,3052,1,0,0,1,0,Housing Support Services,2,1,High level of accessibility,NaN,NaN,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,0
2,3,Common Ground Supportive Housing,144.958496,-37.802791,660-674 Elizabeth Street Carlton 3000,Carlton,3000,1,0,1,0,0,Housing Support Services,2,1,High level of accessibility,NaN,NaN,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,0
3,4,State Library of Victoria,144.965304,-37.809768,304-328 Swanston Street Melbourne City 3000,Melbourne City,3000,1,1,1,0,0,Library,1,1,Moderate level of accessibility,NaN,NaN,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,0
4,5,6th Melbourne Scout Hall,144.928058,-37.791918,55 McCracken Street Kensington 3031,Kensington,3031,1,0,1,1,0,Community Space,2,2,High level of accessibility,NaN,NaN,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,9:00:00,17:00:00,0


In [4]:
list(df.columns)

['id',
 'station_name',
 'longitude',
 'latitude',
 'address',
 'suburb',
 'postcode',
 'if_charger_working',
 'usb_a',
 'usb_c',
 'micro_usb',
 'plug_only',
 'establishment_type',
 'if_wifi',
 'if_bathroom',
 'access_type',
 'other_amenities',
 'star_rating',
 'mon_open',
 'mon_close',
 'tue_open',
 'tue_close',
 'wed_open',
 'wed_close',
 'thu_open',
 'thu_close',
 'fri_open',
 'fri_close',
 'sat_open',
 'sat_close',
 'sun_open',
 'sun_close',
 'if_24h']

In [0]:
df2 = df.groupby("establishment_type", as_index=False ).count()[["establishment_type","id"]]
df2.rename({"id": "est_count"}, axis='index')
df2

,establishment_type,id
0,Community Centre,4
1,Community Space,21
2,Housing Support Services,28
3,Library,130
4,Other Support Services,5
5,Restaurant,30
6,Train Station,8


In [0]:
df3 = df.groupby("postcode", as_index=False ).count()[["postcode","id"]]
df3[df3.id > 5]

,postcode,id
4,3000,23
7,3004,6
21,3031,9
32,3052,9
97,3182,6


time: 20.7 ms


**The section contains extra codes that could possibly be useful for Iteration 2**

In [0]:
#List of suburbs in Melbourne
file_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/Melbourne_LGAS.csv"
df = pd.read_csv(file_path)

regex = ["City of", "shire of"]

df["main"] = df["main"].str.replace(r"\(.*\)","").str.replace(r"\[.*\]","")
df["lga"] = df["main"].apply(lambda x: x if any( l in x for l in lga) else None)
df["lga"].fillna(method='ffill', inplace=True)
df = df[df['main'] != df['lga']]
df["main"] = df["main"].str.strip()
df['postcode'] = df["main"].apply(lambda x: x.strip()[len(x)-4:] if x.strip()[len(x)-4:].isdigit() else None)
df["postcode"].fillna(method='ffill', inplace=True)

#check length of postcode column
#df["postcode"].apply(lambda x: len(x)).unique()

#https://stackoverflow.com/questions/40716571/pandas-replace-part-of-a-column-with-another-column
df["main"] = df.apply(lambda x: x["main"].replace(x["postcode"],""), axis =1).str.strip()

time: 18.2 ms


In [6]:
file_path = "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/For Upload/04272020_upload_v3.csv"
df = pd.read_csv(file_path)
#view file structure

#get list of columns
cols_db = list(df.columns)
print(cols_db)

['id', 'station_name', 'longitude', 'latitude', 'address', 'suburb', 'postcode', 'if_charger_working', 'usb_a', 'usb_c', 'micro_usb', 'plug_only', 'establishment_type', 'if_wifi', 'if_bathroom', 'access_type', 'other_amenities', 'star_rating', 'mon_open', 'mon_close', 'tue_open', 'tue_close', 'wed_open', 'wed_close', 'thu_open', 'thu_close', 'fri_open', 'fri_close', 'sat_open', 'sat_close', 'sun_open', 'sun_close', 'if_24h']


In [0]:
#this is to save the file as csv for additional external analysis in excel
#df.to_csv(r'melbPostCode_data.csv', index = False)
#mount the drive in google collabb

#https://stackoverflow.com/questions/53898836/export-dataframe-as-csv-file-from-google-colab-to-google-drive
#!cp melbPostCode_data.csv "/content/drive/My Drive/B3_FIT5120FIT5122/Iteration 1/Dataset/Python Code/"

time: 2.51 s


In [0]:

#This is a comprehensive search for all possible mcdonald's locations in Melbourne using the address format as McDonald's + <suburb> + <postcode>
def find_maccas(add):
  temp_add = locator.geocode(add).address
  if "McDonald's" in temp:
      lat = locator.geocode(add).latitude
      lon = locator.geocode(add).longitude
      address = temp_add
  else:
    location_all = ["None","None"]
    address = "remove"
  return lat, lon, address


def fullLoc(add):
  full_add = locator.geocode(add)
  #get full name of address
  temp_add = full_add.address

  #get nested adress fields to get suburb and and post code data
  if "McDonald's" in temp_add:
    address_fields = dict(full_add.raw).get("address")
    lat = full_add.geocode(add).latitude
    lon = full_add.geocode(add).longitude
    suburb = address_fields.get("suburb")
    postcode = address_fields.get("postcode")
    county = address_fields.get("county")
    address = full_add.get("display_name").replace(county,"").replace("Australia","").replace("Victoria","").replace(",","").replace("  "," ")
  
  else:
    lat = "None"
    lon = "None"
    suburb = "None"
    postcode = "None"
    address = "None"

time: 22.7 ms
